In [ ]:
pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-f2bslw9e/unsloth_a8f3f5f1738b4637ae9cdac465ca8dd4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-f2bslw9e/unsloth_a8f3f5f1738b4637ae9cdac465ca8dd4
  Resolved https://github.com/unslothai/unsloth.git to commit 79a2112ca4a775ce0b3cb75f5074136cb54ea6df
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from torch import __version__;
from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.0.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton

In [ ]:
from unsloth import FastLanguageModel
import torch

In [ ]:
max_sequence_length = 2048
dtype = None
load_in_4bit = True


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length= max_sequence_length,
    dtype= dtype,
    load_in_4bit= load_in_4bit,
)

==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

In [ ]:
#PEFT Model, LORA Adapter
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)

In [ ]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("lighteval/MATH", "all")

In [ ]:
def format_for_alpaca(row):
  instruction = f"Solve the following {row['type']} problem:"
  input_text = f"Problem : {row['problem']}\nDifficulty : {row['level']}"
  output_text = row['solution']
  return {
      "instruction" : instruction,
      "input" : input_text,
      "output" : output_text
  }

formatted_data = ds.map(format_for_alpaca)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [ ]:

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

In [ ]:
final_formatted_ds = formatted_data.map(
    formatting_prompts_func, batched = True
)

In [ ]:
trainset = final_formatted_ds['train']
testset = final_formatted_ds['test']

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model ,
    tokenizer = tokenizer,
    train_dataset = trainset,
    eval_dataset = testset,
    dataset_text_field = "text",
    max_seq_length = max_sequence_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim  = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),

)

Map (num_proc=2):   0%|          | 0/7500 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/5000 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,500 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 13,631,488


Step,Training Loss
1,1.492900
2,1.560100
3,1.623900
4,1.375000
5,1.415900
6,1.322900
7,1.149300
8,1.252500
9,1.066700
10,0.996900


In [ ]:
from unsloth.chat_templates import get_chat_template

In [ ]:
FastLanguageModel.for_inference(model)

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3",
    mapping = {"role" : "from",
               "content" : "value",
               "user" : "human",
               "assistant" : "gpt"})

messages = [
    {"from": "human",
     "value" : "Explain Bayes Theorem"}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True,
    return_tensors ="pt"
).to('cuda')

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(input_ids = inputs,
                   streamer = text_streamer,
                   max_new_tokens = 1000,
                   use_cache = True)

<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Explain Bayes Theorem<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Bayes Theorem is a formula that describes the relationship between probabilities. It can be expressed as P(A|B) = P(B|A)P(A)/P(B), where P(A|B) is the probability of event A occurring, given that event B has occurred, P(B|A) is the probability of event B occurring, given that event A has occurred, P(A) is the probability of event A occurring, and P(B) is the probability of event B occurring.

The theorem is named after the Reverend Thomas Bayes, an 18th-century English minister who developed the theorem in a paper that was published posthumously in 1763. The theorem is often used in statistics and probability theory, and is a fundamental tool in Bayesian inference, which is a method of statistical inference that uses Bayes' Theorem to update beliefs based on new evidence.

The theorem can be applied in a variety of situations, such as in the field of 

In [ ]:
model.save_pretrained_gguf('model' , tokenizer)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.68 out of 12.67 RAM for saving.


100%|██████████| 32/32 [02:10<00:00,  4.09s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be ./model/unsloth.Q8_0.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:h

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: ./model/unsloth.Q8_0.gguf
Unsloth: Saved Ollama Modelfile to model/Modelfile


In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/model/unsloth.Q8_0.gguf /content/drive/MyDrive/
!cp /content/model/Modelfile /content/drive/MyDrive/

In [ ]:
#from google.colab import files
#files.download('/content/model/unsloth.Q8_0.gguf')